### Imports

In [1]:
%run gpt_content_creator.ipynb
%run gpt_subtopic_creator.ipynb
%run gpt_mindmap_creator.ipynb

import openai
from gpt import GPT
from gpt import Example

### Function: Creates html body wrapper
Creates the html body wrapper around "section" parts

In [2]:
def pack_sections_in_body(sections_list):
    body_start = "<body class=\"jp-Notebook\"><div class=\"reveal\"><div class=\"slides\">"
    body_end = "</div></div></body>"

    sections = "".join(sections_list)

    return body_start + sections + body_end

### Function: Creates jupyter notebook presentation
Creates html jupyter notebook presentation code

In [3]:
def create_markdown_presentation(sections_list, file_name):
    # Static Head Code (Start)
    with open("ressources/markdown_html_header.txt", encoding="utf8") as head:
        header = head.read()

    # GPT generated body code
    with open("ressources/markdown_script_part.txt", encoding="utf8") as end:
        ending = end.read()

    # Static Script Code (Ending)
    with open(file_name + ".html", "w",  encoding="utf8") as html:
        html.write(header)
        html.write(pack_sections_in_body(sections_list))
        html.write(ending)
    
    print("File created!")

### OpenAI Key
Get the OpenAI Key from the txt file

In [4]:
# Openai key
with open("openai_key.txt") as file:
    key = file.read()
    openai.api_key = key

### Model Training
A GPT Model is beeing created and then trained with Examples.

The goal for the model is to create html code from text to be used in a html presentation

In [5]:
# GPT model to create html code from heading and bulletpoints
gpt_markdown_creation = GPT(engine="davinci", temperature=.5, max_tokens=150)

gpt_markdown_creation.add_example(Example("Heading: Cat Text: The cat (Felis catus) is a domesticated species of small carnivorous mammal. It is the only domesticated species in the family Felidae.",
                                       "<section><div class=\"jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput \" data-mime-type=\"text/markdown\"><h1>Cat</h1><ul><li>The cat (Felis catus) is a domesticated species of small carnivorous mammal.</li><li>It is the only domesticated species in the family Felidae.</li></ul></div></section>"
                                        ))

gpt_markdown_creation.add_example(Example("Heading: PyCharm Text: PyCharm is an integrated development environment used in computer programming. It is developed by the Czech company JetBrains.",
                                          "<section><div class=\"jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput \" data-mime-type=\"text/markdown\"><h1>PyCharm</h1><ul><li>PyCharm is an integrated development environment used in computer programming.</li><li>It is developed by the Czech company JetBrains.</li></ul></div></section>"
                                        ))

gpt_markdown_creation.add_example(Example("Heading: python Text: Python is a high-level programming language. It is a general-purpose language. It is a high-level language. It is a widely used language.",
                                          "<section><div class=\"jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput \" data-mime-type=\"text/markdown\"><h1>python</h1><ul><li>Python is a high-level programming language.</li><li>It is a general-purpose language.</li><li>It is a high-level language.</li><li>It is a widely used language.</li></ul></div></section>"
                                        ))

gpt_markdown_creation.add_example(Example("Heading: Etymology and naming Text: The origin of the English word cat is thought to be the Late Latin word cattus, which was first used at the beginning of the 6th century.",
                                          "<section><div class=\"jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput \" data-mime-type=\"text/markdown\"><h1>Etymology and naming</h1><ul><li>The origin of the English word cat is thought to be the Late Latin word cattus, which was first used at the beginning of the 6th century.</li></ul></div></section>"
                                        ))

### Get user input for topic

In [6]:
prompt = input("Topic: ")

Topic: enterprise


### Generate Subtopics
Uses the trained subtopic model from the gpt_subtopic_creator.ipynb file to gererate subtopics

In [7]:
# Get subtopics from topic
subtopics = create_subtopics_from_topic(prompt)
subtopics = subtopics.split(", ")

# Convert text to list and ad the main topic at first place
subtopics_list = [prompt]
for i in subtopics:
    subtopics_list.append(i.strip())

# Limit the subtopic count
while True:
    num = input("Max number of slides: ")
    try:
        int(num)
        break
    except:
        print("Must be a number!")

if len(subtopics_list) > int(num):
    subtopics_list = subtopics_list[:int(num)]

# Print summary of generated subtopics
print("GPT-3 generated subtopics:\n", subtopics_list)


mindmap_topics = ", ".join(subtopics_list)

Max number of slides: 6
GPT-3 generated subtopics:
 ['enterprise', 'business', 'company', 'corporation', 'profit', 'market']


### Generate Text for Subtopics
Goes through every subtopic and uses the trained gpt model to generate text for it.

In [8]:
# Get text for every subtopic
section_list = []
for topic in subtopics_list:

    if prompt not in topic:
        topic = prompt + " " + topic

    print("\n" + topic)
    print("---------------")

    content = create_text_from_topic(topic)

    # Clean up content output
    content.replace("\"", "")
    content.replace("output:", "")

    # Convert topic and bulletpoints to format for the html GPT model
    content_input = "Heading: " + topic + " Text: " + content

    # Create html Code from GPT Model and create presentation
    output = gpt_markdown_creation.submit_request(content_input)

    # clean up output code
    section_html = output.choices[0].text[8:]
    section_html.replace("output:", "")
    section_html = section_html.strip()
    section_html = section_html.split("</section>")[0] + "</section>"

    print("\nGPT-3 generated html:\n" + section_html)
    section_list.append(section_html)


enterprise
---------------
GPT-3 generated Text:
An enterprise is a business or organisation. An enterprise is a business or organization, typically of large size.

GPT-3 generated html:
<section><div class="jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput " data-mime-type="text/markdown"><h1>enterprise</h1><ul><li>An enterprise is a business or organisation.</li><li>An enterprise is a business or organization, typically of large size.</li></ul></div></section>

enterprise business
---------------
GPT-3 generated Text:
Enterprise business is the business of enterprise, the organization of people and other resources to achieve a specific goal.

GPT-3 generated html:
<section><div class="jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput " data-mime-type="text/markdown"><h1>enterprise business</h1><ul><li>Enterprise business is the business of enterprise, the organization of people and other resources to achieve a specific goal.</li></ul></div></section>

enterprise 

### Generate Html Mindmap for Subtopics
Creates a mindmap with each subtopic

In [9]:
# Ask if mindmap should be created
ans = input("Create an overview mindmap? [y/n]: ")
if ans.lower() == "y":
    
    # Create mindmap
    print("\nInput for mindmap:\n" + mindmap_topics + "\n")
    mindmap_html = create_mindmap_from_topic(mindmap_topics)
    section_list.insert(0,mindmap_html)

Create a overview mindmap? [y/n]: y

Input for mindmap:
enterprise, business, company, corporation, profit, market

GPT-3 generated mindmap html code:
<section><h1>Overview</h1><div class="mindmap-container"><div class="topic">enterprise</div><ul class="mindmap-subtopics"><li class="mindmap-list">business</li><hr id="line1"><li class="buff"></li><hr id="line2"><li class="mindmap-list">company</li><hr id="line3"><li class="buff"></li><hr id="line4"><li class="mindmap-list">corporation</li><hr id="line5"><li class="buff"></li><hr id="line6"><li class="mindmap-list">profit</li><hr id="line7"><li class="buff"></li><hr id="line8"><li class="mindmap-list">market</li><hr id="line9"><li class="buff"></li></ul></div></section>


### Generate the html file
Used the defined function to generate the html file

In [10]:
create_markdown_presentation(section_list, prompt + "_gpt_presentation")

File created!
